In [1]:
from astral import LocationInfo
from astral.sun import sun
from datetime import date

city = LocationInfo(
    name="Manus",
    region="PNG",
    timezone="Pacific/Port_Moresby",  # Critical for local time
    latitude=-2.08,
    longitude=147.42
)

s = sun(city.observer, date=date.today(), tzinfo=city.timezone)

print("Sunrise:", s['sunrise'])  # Local time
print("Sunset:", s['sunset'])    # Local time

Sunrise: 2025-07-11 06:15:47.230650+10:00
Sunset: 2025-07-11 18:15:54.992329+10:00


In [5]:
# apply the zone offset to UTC and remove the timezone info
print("Sunrise UTC:", s['sunrise'].astimezone(tz=None))

Sunrise UTC: 2025-07-10 16:15:47.230650-04:00


In [ ]:
from pvlib.location import Location
import pandas as pd

loc = Location(latitude=-2.08, longitude=147.42, tz="Pacific/Port_Moresby", altitude=0)
times = pd.date_range("2025-07-11", periods=1, freq='D', tz=loc.tz)
solar_pos = loc.get_sun_rise_set_transit(times)



2025-07-10 06:15:35.674678272+10:00


/var/folders/bt/kswp83jx5mj4bn1bjkh9g4rsg_b7ks/T/ipykernel_2958/1264533105.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(solar_pos['sunrise'][0])


In [ ]:
import pandas as pd 
import numpy as np
from datetime import datetime

date_range_twp = pd.date_range(start='2000-01-01', end='2015-01-01', freq='D').date


dft_twp_sunset_sunrise = pd.DataFrame(date_range_twp,columns=['date'])


def calc_pvlib_sunrise_sunset(date):
    loc = Location(latitude=-2.06, longitude=147.42, tz="UTC", altitude=4)
    times = pd.date_range(date, periods=1, freq='D', tz=loc.tz)
    solar_pos = loc.get_sun_rise_set_transit(times)
    sunrise = pd.Timestamp(solar_pos['sunrise'].iloc[0]).tz_localize(None)
    sunset = pd.Timestamp(solar_pos['sunset'].iloc[0]).tz_localize(None)
    return pd.Series([sunrise, sunset], index=['sunrise(UTC)', 'sunset(UTC)'])


sunset_sunrise = dft_twp_sunset_sunrise['date'].apply(calc_pvlib_sunrise_sunset)

dft_twp_sunset_sunrise = pd.concat([dft_twp_sunset_sunrise, sunset_sunrise], axis=1)


In [ ]:


dft_twp_sunset_sunrise.to_csv('sunrise_sunset_twp.csv', index=False)

,date,sunrise(UTC),sunset(UTC)
0,2000-01-01,1999-12-31 20:06:36.399570688,2000-01-01 08:20:41.957186048
1,2000-01-02,2000-01-01 20:07:05.671954176,2000-01-02 08:21:09.383139968
2,2000-01-03,2000-01-02 20:07:34.699391488,2000-01-03 08:21:36.403096320
3,2000-01-04,2000-01-03 20:08:03.447401216,2000-01-04 08:22:02.984558208
4,2000-01-05,2000-01-04 20:08:31.881998208,2000-01-05 08:22:29.095588096
...,...,...,...
5475,2014-12-28,2014-12-27 20:04:46.771648768,2014-12-28 08:18:57.582484992
5476,2014-12-29,2014-12-28 20:05:16.620607488,2014-12-29 08:19:26.208547072
5477,2014-12-30,2014-12-29 20:05:46.309311744,2014-12-30 08:19:54.505681664
5478,2014-12-31,2014-12-30 20:06:15.806437632,2014-12-31 08:20:22.443540480
